In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from collections import Counter

df=pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")

In [3]:
# View Shape of Dataset
df.shape

(8807, 12)

In [4]:
# View dataset columns
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [5]:
#Null values for each column
print("Null values :\n")
for col in df.columns:
    null_perc=(df[col].isnull().sum()/len(df[col]))*100
    if null_perc>0:
        print(f"{col}:  {null_perc:.2f}%")


Null values :

director:  29.91%
cast:  9.37%
country:  9.44%
date_added:  0.11%
rating:  0.05%
duration:  0.03%


In [6]:
df["date_added"]=pd.to_datetime(df["date_added"])

df["year_added"]=df["date_added"].dt.year

df["month_added"]=df["date_added"].dt.month

In [7]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",2021.0,9.0
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021.0,9.0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021.0,9.0
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021.0,9.0
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021.0,9.0


# 1. Type of Content on Netflix

In [8]:
movies=df[df["type"]=="Movie"]
shows=df[df["type"]=="TV Show"]

count_df=df["type"].value_counts().reset_index().rename(columns={"type":"count","index":"type"})

pie_=go.Pie(labels=count_df["type"], values=count_df["count"],pull=[0.1,0],marker=dict(colors=["#34e8eb", "#eb9e34"]))

go.Figure(data=[pie_])
go.Pie()

Pie()

# Duration of Movies

In [9]:
duration=df[df["type"]=="Movie"]["duration"].fillna(method="backfill").apply(lambda x: int(x.split(" ")[0])).reset_index()

In [10]:
less_than_100=len(duration[duration["duration"]<=100])
_100_to_120=len(duration[(duration["duration"]>100) & (duration["duration"]<120)])
more_than_120=len(duration[duration["duration"]>=120])

labels=["Less Than 100 Mins","Between 100 to 120 Mins","More Than 120 Mins"]

values=[less_than_60,_60_to_120,more_than_120]

trace=go.Bar(x=values, y= labels, orientation="h",width=0.5, marker=dict(color="#34ebab"))
go.Figure(trace)

NameError: name 'less_than_60' is not defined

# Number of Seasons

# Content Added on the Netflix over the Years

In [ ]:
trend_movies=movies["year_added"].value_counts().reset_index().rename(columns={"year_added":"count","index":"year_added"}).sort_values("year_added")

trend_shows=shows["year_added"].value_counts().reset_index().rename(columns={"year_added":"count","index":"year_added"}).sort_values("year_added")

graph_movies=go.Scatter(x=trend_movies["year_added"],y=trend_movies["count"],marker=dict(color="#34e8eb"), name="Movies")

graph_shows=go.Scatter(x=trend_shows["year_added"],y=trend_shows["count"],marker=dict(color="#eb9e34"), name="TV Shows")

go.Figure([graph_movies,graph_shows])

In [ ]:
feature="release_year"
release_movies=movies[feature].value_counts().reset_index().rename(columns={feature:"count","index":feature}).sort_values(feature)
release_shows=shows[feature].value_counts().reset_index().rename(columns={feature:"count","index":feature}).sort_values(feature)

bar_movies=go.Bar(x=release_movies[feature],y=release_movies["count"],marker=dict(color="#34e8eb"), name="Movies")
bar_shows=go.Bar(x=release_shows[feature],y=release_shows["count"],marker=dict(color="#eb9e34"), name="TV Shows")
go.Figure([bar_movies,bar_shows])

# Content from Different contries:

In [ ]:
feature='country'
feature_count=df[feature].value_counts()

tabs = Counter(dict(feature_count)).most_common(15)

labels = [_[0] for _ in tabs][::-1]
values = [_[1] for _ in tabs][::-1]
trace = go.Bar(y=labels, x=values, orientation="h", name="", marker=dict(color="#34b1eb"))

layout = go.Layout(title="Countries with most content", width=500, legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure([trace], layout=layout)
fig.show()

# Top categories in content

In [11]:
category_count=Counter(", ".join(df["listed_in"]).split(", "))
top_category=category_count.most_common(20)
labels=list(dict(top_category).keys())
labels.reverse()
values=list(dict(top_category).values())
values.reverse()

bar=go.Bar(y=labels, x=values, orientation="h", marker=dict(color="#34b1eb"))
go.Figure(bar)

In [12]:
Counter(", ".join(df["country"].dropna()).split(",")).most_common(6)

[(' United States', 3689),
 (' India', 1046),
 (' United Kingdom', 806),
 (' Canada', 445),
 (' France', 393),
 (' Japan', 318)]

In [13]:
def country_trace(country, flag="movie"):
    df["flag"]=df["country"].fillna("").apply(lambda x: 1 if country.lower() in x.lower() else 0)
    frame=df[df["flag"]==1]
    if flag=="movie":
        frame=frame[frame["type"]=="Movie"]
    else:
        frame=frame[frame["type"]=="TV Show"]
        
    cast=", ".join(frame["cast"].dropna()).split(", ")
    
    cast_count= dict(Counter(cast).most_common(15))
    labels=list(cast_count.keys())
    labels.reverse()
    
    values=list(cast_count.values())
    values.reverse()
    
    trace=go.Bar(x=values, y= labels, orientation="h", marker=dict(color="#34b1eb"))
    return trace

# Top Actors in India on Netflix with Most Movies

In [14]:
country="India"
go.Figure(country_trace(country,flag= "movie"))

# Top Actors in United States on Netflix with Most TV Shows


In [15]:
country="United States"
go.Figure(country_trace(country,flag= "TV Show"))

# Top 10 Movie Directors from India with most content

In [16]:
directors=dict(df[df["country"]=="India"]["director"].dropna().value_counts())

labels=list(directors.keys())[:10]
labels.reverse()

values=list(directors.values())[:10]
values.reverse()

trace=go.Bar(x=values, y= labels, orientation="h", marker=dict(color="#ebba34"))
go.Figure(trace)

# Top 10 Movie Directors from US with most content

In [17]:
directors=dict(df[df["country"]=="United States"]["director"].dropna().value_counts())

labels=list(directors.keys())[:10]
labels.reverse()

values=list(directors.values())[:10]
values.reverse()

trace=go.Bar(x=values, y= labels, orientation="h", marker=dict(color="#ebba34"))
go.Figure(trace)